In [1]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
from random import randint
from matplotlib import pyplot as plt

In [2]:
from common import get_full_data
# load data
def steel_strength():
    df = pd.read_csv("dataset/steel_strength.csv")
    # get dependent and independent features
    X=df.iloc[:,1:-3]
    y=df.iloc[:,-2]
    return get_full_data(X,y)

In [3]:
from xgboost import XGBRegressor
X,y = steel_strength()

# for high-dimensional data use `gpu` for device if you have one
special_model = XGBRegressor(device='cpu')

In [4]:
from sklearn.model_selection import RandomizedSearchCV
from common import XGB_search_params

params = XGB_search_params()
state = randint(0,1000)
search = RandomizedSearchCV(
    special_model,
    params,
    n_iter=500,
    cv=5,
    random_state=state,
    n_jobs=-1,
)
search.fit(X,y)
special_model=search.best_estimator_

In [5]:
# do repeated stratified k-fold cross-validation with classification report
from sklearn.model_selection import RepeatedKFold, RepeatedStratifiedKFold, cross_val_score
from common import cross_val_classification_report, cross_val_score_mean_std

special_model.device='cpu'

cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=50)
r2_scoring = metrics.make_scorer(metrics.r2_score)
print("r2 scoring")
cross_val_score_mean_std(cross_val_score(special_model,X,y,cv=cv,scoring=r2_scoring),y.name)

r2 scoring
-----------tensile strength-----------
Mean  0.8365877854992385
Std  0.06572885995838901


In [6]:
# New method
from common import find_outliers, negate

outliers, score = find_outliers(
    X.to_numpy(),
    y,
    special_model,
    outlier_remove_partition=0.05,
    evaluate_loss=metrics.mean_squared_error,
    pred_loss=metrics.mean_squared_error,
    plot=False
)
X_clean = [x for i,x in enumerate(X.to_numpy()) if i not in outliers]
y_clean = [y_ for i,y_ in enumerate(y) if i not in outliers]

r2_scoring = metrics.make_scorer(metrics.r2_score)
print("r2 score")
cross_val_score_mean_std(cross_val_score(special_model,X_clean,y_clean,cv=cv,scoring=r2_scoring),y.name)


r2 score
-----------tensile strength-----------
Mean  0.8853371995060441
Std  0.055296365202510404


In [7]:
# z-score method
from scipy import stats
data = pd.concat([X,y],axis=1)
z = np.abs(stats.zscore(data))
threshold = 3
data_clean = data[(z < threshold).all(axis=1)]

X_clean=data_clean.iloc[:,:-2]
y_clean=data_clean.iloc[:,-1]

r2_scoring = metrics.make_scorer(metrics.r2_score)
print("r2 score")
cross_val_score_mean_std(cross_val_score(special_model,X_clean,y_clean,cv=cv,scoring=r2_scoring),y.name)

r2 score
-----------tensile strength-----------
Mean  0.8262127045515743
Std  0.09445724604889805


In [8]:

import pandas as pd

# Assuming 'data' is a Pandas DataFrame
Q1 = data.quantile(0.15)
Q3 = data.quantile(0.85)
IQR = Q3 - Q1
data_clean = data[~((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))).any(axis=1)]

X_clean=data_clean.iloc[:,:-2]
y_clean=data_clean.iloc[:,-1]

r2_scoring = metrics.make_scorer(metrics.r2_score)
cross_val_score_mean_std(cross_val_score(special_model,X_clean,y_clean,cv=cv,scoring=r2_scoring),y.name)

-----------tensile strength-----------
Mean  0.8436396395374473
Std  0.09318957218688252


In [9]:
from sklearn.ensemble import IsolationForest


clf = IsolationForest(random_state=50)
outliers_pred=clf.fit_predict(data)

data_clean = data[outliers_pred==1]

X_clean=data_clean.iloc[:,:-2]
y_clean=data_clean.iloc[:,-1]

r2_scoring = metrics.make_scorer(metrics.r2_score)
cross_val_score_mean_std(cross_val_score(special_model,X_clean,y_clean,cv=cv,scoring=r2_scoring),y.name)

-----------tensile strength-----------
Mean  0.8181715818935182
Std  0.08521800031668554
